In [3]:
from collections import Counter
import itertools
import json
import math
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas
import pennylane as qml
from pennylane import numpy as np
from pennylane import qaoa
from scipy.optimize import minimize
import random
import time

In [5]:
# load data 
library_df = pandas.read_csv("example_data_18_6.csv")

In [6]:
# calculate optimal values and set constants
color_o = 0
shape_o = 0
components_o =0
library_data = []
samples_per_region = None

for region in library_df["region"].unique():
    nodes = library_df.loc[library_df['region'] == region]  
    if not samples_per_region:
        samples_per_region = len(library_df.loc[library_df['region'] == region])
        
    library_data.append(nodes)
    color_o += library_df['color'].min()
    shape_o += library_df['shape'].min()
    components_o += library_df['components'].min()

region_num = len(library_df['region'].unique())

library_df['wire'] = range(len(library_df))

num_qubits = len(library_df)

In [8]:
def region_hamiltonian_quadratic(df):
    """
    Construct a Hamiltonian representing the one fragment per region constraint. 
    
    Parameters:
    - df (pandas.DataFrame): A dataframe containing the problem data.

    Returns:
    - pennylane.Hamiltonian: A Hamiltonian object that encodes the penalty terms for 
      deviating from the target property value. The Hamiltonian consists of weighted Z 
      Pauli operators, where the weights depend on the property values and the desired 
      constraint.
    """
    
    coeffs = []
    observables = []
    
    # Iterate over each unique region/group in the DataFrame
    for i, region in enumerate(df['region'].unique()):
        # Get the positions of the nodes in the current region
        nodes_in_region = df.loc[df['region'] == region]['wire'].to_list()
        
        # Linear penalty: Encourage selecting a single node (reward for selection)
        for j in range(len(nodes_in_region)):
            coeffs.append(-1)
            observables.append(qml.PauliZ(nodes_in_region[j]))
        
        # Quadratic penalty: Discourage selecting multiple nodes (penalize multiple selections)
        for j in range(len(nodes_in_region)):
            for k in range(j+1, len(nodes_in_region)):
                coeffs.append(2)
                observables.append(qml.PauliZ(nodes_in_region[j]) @ qml.PauliZ(nodes_in_region[k]))

        # Higher-order penalty terms
        num_nodes = len(nodes_in_region)
        coeffs.append(num_nodes)
        observables.append(qml.Identity(0))  # Just an identity term to match the Hamiltonian structure

        for node in nodes_in_region:
            coeffs.append(-2)
            observables.append(qml.PauliZ(node))
            
            for other_node in nodes_in_region:
                if node != other_node:
                    coeffs.append(1)
                    observables.append(qml.PauliZ(node) @ qml.PauliZ(other_node))

    return qml.Hamiltonian(coeffs, observables)

In [9]:
def property_hamiltonian_from_df(df, property_name, property_constraint):
    """
    Construct a Hamiltonian representing the constraint on a specific  property 
    across a set of feasible solutions, given a target constraint value. 
    
    This function creates a Hamiltonian that penalizes deviations from a desired property 
    value. It is useful in quantum optimization problems where one needs to find a 
    configuration of records that meets certain criteria.

    Parameters:
    - df (pandas.DataFrame): A dataframe containing the problem data.
    - property_name (str): The name of the column in the dataframe `df` that contains the 
    property values to be constrained.
    - property_constraint (float): The target value for the property that the 
      Hamiltonian should enforce.

    Returns:
    - pennylane.Hamiltonian: A Hamiltonian object that encodes the penalty terms for 
      deviating from the target property value. The Hamiltonian consists of weighted Z 
      Pauli operators, where the weights depend on the property values and the desired 
      constraint.

    Example:
    >>> df = pd.DataFrame({'molecule_id': [1, 2, 3],
                           'property_x': [1.2, 0.8, 1.5]})
    >>> hamiltonian = property_hamiltonian_from_df(df, 'property_x', 1.0)
    >>> print(hamiltonian)
    """
    
    # Initialize lists to hold the coefficients and observables for the Hamiltonian
    coeffs = []
    observables = []
    
    # Extract the list of property values from the dataframe for the given  property
    property_list = df[property_name].to_list()
    
    # Iterate over each property value to construct the Hamiltonian terms
    for index, property_value in enumerate(property_list):
        # Add a term penalizing the deviation from the target constraint for each property
        coeffs.append(-2 * property_value * property_constraint)
        
        # Apply Z Pauli operator to the corresponding qubit
        observables.append(qml.PauliZ(index))  
        
        # Add terms for every pair of properties to account for their interaction
        for index2, property_value_2 in enumerate(property_list):
            if index < index2:  # Ensure that each pair is only considered once
                coeffs.append(2 * property_value * property_value_2)
                
                # Apply Z Pauli operators to the pair of qubits
                observables.append(qml.PauliZ(index) @ qml.PauliZ(index2))  
               
    # Create and return the Hamiltonian object with the specified coefficients and observables
    return qml.Hamiltonian(coeffs, observables)

In [10]:
# Construct Hamiltonians 
H_regions = region_hamiltonian_quadratic(library_df)

H_color = property_hamiltonian_from_df(library_df, "color", color_o)
H_shape = property_hamiltonian_from_df(library_df, "shape", shape_o)
H_components = property_hamiltonian_from_df(library_df, "components", components_o)
# Assuming equal importance for simplicity
alpha_regions = 1000
alpha_color = 100
alpha_shape = 1
alpha_components = 100

# Total Hamiltonian
H_total = alpha_regions * H_regions + alpha_color * H_color + alpha_components * H_components #+ alpha_shape * H_shape 

In [11]:
def create_mixer_hamiltonian(library_df):
    """
    Creates a mixer Hamiltonian for use in a QAOA circuit.

    The mixer Hamiltonian is composed of Pauli-X operations applied to each qubit in the system.
    This type of mixer is often used in QAOA to introduce transitions between different computational
    basis states, facilitating exploration of the solution space in combinatorial optimization problems.

    The coefficients of the Hamiltonian are all set to 1, indicating an equal weight for the influence
    of each Pauli-X operation on its respective qubit.

    Parameters:
    - library_df: A data structure (implied to be a DataFrame or similar, based on the name) whose length
      determines the number of qubits in the system. Each qubit will have a Pauli-X operation applied.

    Returns:
    - mixer_h: A qml.Hamiltonian object representing the mixer Hamiltonian with Pauli-X operations
      applied to each qubit. The Hamiltonian is used to drive transitions between states during the
      mixing phase of the QAOA algorithm.

    Example usage:
    >>> library_df = [data representing your problem instance]
    >>> mixer_h = create_mixer_hamiltonian(library_df)
    >>> print(mixer_h)
    """
    

    mixer_h = qml.Hamiltonian(coeffs=[1]*len(library_df), observables=[qml.PauliX(i) for i in library_df['wire']])
    return mixer_h

In [13]:
# "Build the Circuit
complex_dtype = np.complex128
# Define the device
                  
dev = qml.device('lightning.gpu', wires=num_qubits, c_dtype = complex_dtype)
#prepare_optimal_state()
cost_h = H_total
 
mixer_h = create_mixer_hamiltonian(library_df)
def qaoa_circuit(params):
    
    for i in range(p):
        qaoa.cost_layer(params[0][i], cost_h)
        qaoa.mixer_layer(params[1][i], mixer_h)
    return qml.expval(cost_h)


qnode = qml.QNode(qaoa_circuit, dev, diff_method="adjoint") 

In [26]:
# start parameter training

p = 2
opt = qml.AdamOptimizer(stepsize=0.001)
num_iterations = 100
np.random.seed(0)
init_params = np.random.rand(2, p) * np.pi

adam_values = []

params = init_params
for step in range(num_iterations):
    print(step)
    params = opt.step(qnode, params)
   
    adam_values.append(qnode(params))
    if step % 10 == 0:
        print(f'Step {step}: Cost = {adam_values[-1]}')
        
optimized_params_reshaped = params

0
Step 0: Cost = 130957.35804012176
1
2
3
4
5
6
7
8


KeyboardInterrupt: 

In [28]:
# uncoment to skip training and use default parameters

# optimized_params_reshaped = np.array([[1.7314061 , 2.23572227],
#         [2.12054033, 1.94380265]])

In [29]:
# run the circuit with the trained parameters

start_time = time.time()

# sample the circuit 
p = 2 
num_shots = 1000

# this container is setup to run on GPU
# run with lightnig.qubit instead to run on CPU
dev = qml.device('lightning.gpu', wires=num_qubits, shots=num_shots)

# use this device instead to run on IBM instead of the simulator 
# dev = qml.device('qiskit.ibmq', #'qiskit.aer',# 
#                  overwrite=True,
#                  wires=len(library_df), 
#                  backend='ibm_osaka', #'ibmq_qasm_simulator', 
#                  ibmqx_token="xxxx")

cost_h = H_total
mixer_h = create_mixer_hamiltonian(library_df)

def sampling_circuit(params):
    
    for i in range(p):
        qaoa.cost_layer(params[0][i], cost_h)
        qaoa.mixer_layer(params[1][i], mixer_h)

    return [qml.sample(qml.PauliZ(i)) for i in range(num_qubits)]

qnode = qml.QNode(sampling_circuit, dev)#, diff_method="spsa") 
shots_results = qnode(optimized_params_reshaped)

# Initialize a list to hold the optimal_nodes for each shot
optimal_nodes = []
 

In [30]:
 # Reconstruct the bitstrings for each shot
for shot_index in range(num_shots):   
    shot_indexes = [node_record for node_record in range(num_qubits) if shots_results[node_record][shot_index] == 1]
    optimal_nodes.append(shot_indexes)
    

In [31]:
def top_n_indices_complete_set(values, n):
    """
    Find the indices with the highest values and return the top n values
    """
    
    # Convert the list to a numpy array
    values_array = np.array(values)
    
    # Get the indices that would sort the array in descending order
    sorted_indices = np.argsort(values_array)[::-1]
    
    # Select the top n indices
    top_n_sorted_indices = sorted_indices[:n]
    
    # Sort these indices to maintain the original order
    top_n_indices_in_original_order = [int(i) for i in sorted(top_n_sorted_indices)]
     
    return top_n_indices_in_original_order 

def top_n_indices_per_region(values, n):
    """
    Find the top indices per each region
    """
    
    # Convert the list to a numpy array
    values_array = np.array(values)
    optimal_sample = []
    for region in range(1, region_num + 1): 
        region_min = (region -1)  * samples_per_region
        region_max = region* samples_per_region-1
        choice = np.argmax(values_array[region_min:region_max]).max()
       
        optimal_sample.append(choice+region_min)    
        
    # find the top recurring qubits
    top_n_indices = top_n_indices_complete_set(values, n)
    
    # find the top values not in the optimal sample
    unrepresented_index = [index for index in top_n_indices if index not in optimal_sample]

    # extend the optimal_sample with the other top samples not to exceed the requried number of samples
    optimal_sample.extend(unrepresented_index[:region_num - n])

    optimal_sample.sort()
    top_n_indices_in_original_order =  optimal_sample
    
     
    return top_n_indices_in_original_order 

In [32]:
def calculate_sample_percentage(shot_range_start=None, shot_range_end=None, val_num=3):
    """
    Future exploration:
    combine all of the shots and calulate the percentage of the qubit being
    in an excited state
    """
    simulated_samples = np.array(shots_results)
    # Transpose the array so each row represents a shot
    # transposed_samples = simulated_samples.T
    # Convert samples from -1 (state |0>) and 1 (state |1>) to 0 and 1
    samples = (simulated_samples + 1) / 2

    if shot_range_end: 
        samples = [lst[shot_range_start:shot_range_end] for lst in samples]
    

    # Calculate the percentage of each qubit being in state |1>
    percentage_selected = np.mean(samples , axis=1) * 100

    optimal_sample = []

    optimal_sample = top_n_indices_per_region(percentage_selected.tolist(), val_num)

    return optimal_sample

In [33]:
# using 10 "reads" at a time 
start = 0
optimal_nodes = []
for end in range(0,1000, 10):
    nodes = calculate_sample_percentage(start,end, region_num + 4)
    if len(nodes) < region_num:
        continue
    if nodes in optimal_nodes:
        continue
    optimal_nodes.append(nodes)
    start = end

In [34]:
def prep_combos(optimal_nodes, region_num):
    """
    prepare all possible valid combinations 
    """
    
    # Generate all valid combinations  
    valid_combinations = []
    
    
    # Filter the DataFrame to get only the provided indexes
    provided_df = library_df[library_df['wire'].isin(optimal_nodes)]

    regions = provided_df['region'].unique()
    if len(regions) != region_num:
        print(f"invalid number of regions: {regions}")
        return valid_combinations

    # Organize the indexes into their respective regions
    organized_indexes = provided_df.groupby('region')['wire'].apply(list).to_dict()


    for comb in itertools.combinations(optimal_nodes, region_num):
        region_count = {region: 0 for region in organized_indexes}
        for index in comb:
            region = library_df.loc[library_df['wire'] == index, 'region'].values[0]
            if region in region_count:
                region_count[region] += 1
        if all(count >= 1 for count in region_count.values()):
                valid_combinations.append(comb)
    return valid_combinations

In [35]:
post_processed_optimal_nodes = []
proper_r_groups = 0

for optimal_node_set in optimal_nodes:
    print(len(optimal_node_set))
    if len(optimal_node_set) != region_num:
        #continue
        #if not len(optimal_node_set) < r_group_num + 2:
        #    continue
 
        for pp_set in prep_combos(optimal_node_set, region_num):#list(itertools.combinations(optimal_node_set, r_group_num)):
            #print(pp_set)
            # a way to explore solutions that don't pass the 1 hot constraint
            if pp_set not in post_processed_optimal_nodes:  
                post_processed_optimal_nodes.append(pp_set)

                proper_r_groups += 1
                # Create a new dataframe from the optimal node information
                optimal_df = library_df.loc[pp_set, ['id','region', 'color', 'shape', 'components']]

            
                # Print the new dataframe
                print("Optimization totals:")
                print(f"total color: {optimal_df['color'].sum()}")
                print(f"total shape: {optimal_df['shape'].sum()}")
                print(f"total components: {optimal_df['components'].sum()}")
                print("")
                print("Optimal Node Information:")
                print(optimal_df)
                print("-----")
    else:
        post_processed_optimal_nodes.append(optimal_node_set)
        proper_r_groups += 1

        # Create a new dataframe from the optimal node information
        optimal_df = library_df.loc[optimal_node_set, ['id','region', 'color', 'shape', 'components']]

        #print(library_info["connectivity"])
        # Print the new dataframe
        print("Optimization totals:")
        print(f"total color: {optimal_df['color'].sum()}")
        print(f"total shape: {optimal_df['shape'].sum()}")
        print(f"total components: {optimal_df['components'].sum()}")
        print("")
        print("Optimal Node Information:")
        print(optimal_df)
        print("-----")

6
Optimization totals:
total color: 30
total shape: 22
total components: 19

Optimal Node Information:
    id  region  color  shape  components
1    2       1      4      3           1
4    5       2      2      4           2
7    8       3      5      4           4
9   10       4      5      1           4
13  14       5     10      5           5
16  17       6      4      5           3
-----
8
Optimization totals:
total color: 34
total shape: 22
total components: 22

Optimal Node Information:
    id  region  color  shape  components
0    1       1      5      2           1
3    4       2      1      5           5
6    7       3      9      4           4
9   10       4      5      1           4
13  14       5     10      5           5
16  17       6      4      5           3
-----
Optimization totals:
total color: 33
total shape: 23
total components: 22

Optimal Node Information:
    id  region  color  shape  components
1    2       1      4      3           1
3    4       2      1    

Optimization totals:
total color: 33
total shape: 18
total components: 16

Optimal Node Information:
    id  region  color  shape  components
0    1       1      5      2           1
4    5       2      2      4           2
6    7       3      9      4           4
9   10       4      5      1           4
12  13       5      6      2           2
15  16       6      6      5           3
-----
Optimization totals:
total color: 32
total shape: 18
total components: 18

Optimal Node Information:
    id  region  color  shape  components
0    1       1      5      2           1
5    6       2      1      4           4
6    7       3      9      4           4
9   10       4      5      1           4
12  13       5      6      2           2
15  16       6      6      5           3
-----
Optimization totals:
total color: 32
total shape: 19
total components: 16

Optimal Node Information:
    id  region  color  shape  components
1    2       1      4      3           1
4    5       2      2      4 

6
Optimization totals:
total color: 24
total shape: 19
total components: 16

Optimal Node Information:
    id  region  color  shape  components
0    1       1      5      2           1
3    4       2      1      5           5
7    8       3      5      4           4
10  11       4      1      1           1
12  13       5      6      2           2
15  16       6      6      5           3
-----
6
Optimization totals:
total color: 36
total shape: 22
total components: 22

Optimal Node Information:
    id  region  color  shape  components
0    1       1      5      2           1
3    4       2      1      5           5
6    7       3      9      4           4
9   10       4      5      1           4
13  14       5     10      5           5
15  16       6      6      5           3
-----
8
Optimization totals:
total color: 22
total shape: 18
total components: 18

Optimal Node Information:
    id  region  color  shape  components
0    1       1      5      2           1
3    4       2      1  

Optimization totals:
total color: 24
total shape: 18
total components: 15

Optimal Node Information:
    id  region  color  shape  components
0    1       1      5      2           1
5    6       2      1      4           4
7    8       3      5      4           4
10  11       4      1      1           1
12  13       5      6      2           2
15  16       6      6      5           3
-----
Optimization totals:
total color: 22
total shape: 21
total components: 19

Optimal Node Information:
    id  region  color  shape  components
2    3       1      3      4           4
3    4       2      1      5           5
7    8       3      5      4           4
10  11       4      1      1           1
12  13       5      6      2           2
15  16       6      6      5           3
-----
Optimization totals:
total color: 22
total shape: 20
total components: 18

Optimal Node Information:
    id  region  color  shape  components
2    3       1      3      4           4
5    6       2      1      4 

Optimization totals:
total color: 21
total shape: 19
total components: 15

Optimal Node Information:
    id  region  color  shape  components
1    2       1      4      3           1
5    6       2      1      4           4
7    8       3      5      4           4
10  11       4      1      1           1
12  13       5      6      2           2
16  17       6      4      5           3
-----
Optimization totals:
total color: 20
total shape: 20
total components: 18

Optimal Node Information:
    id  region  color  shape  components
2    3       1      3      4           4
5    6       2      1      4           4
7    8       3      5      4           4
10  11       4      1      1           1
12  13       5      6      2           2
16  17       6      4      5           3
-----
8
Optimization totals:
total color: 23
total shape: 18
total components: 13

Optimal Node Information:
    id  region  color  shape  components
0    1       1      5      2           1
4    5       2      2      

In [36]:
def generate_results(sample_set):
    
    temp_results = []
    
    for sample in sample_set:
        results_data ={}
        optimal_df = library_df.loc[sample, ['id','region', 'color', 'shape', 'components']]
        results_data["name"] = sample
        results_data['color'] = optimal_df["color"].sum()
        results_data['shape'] = optimal_df["shape"].sum()
        results_data['components'] = optimal_df["components"].sum()
        results_data["score"] = (results_data["color"] + results_data["components"]) -12
        temp_results.append(results_data)
        
    results_df = pandas.DataFrame.from_dict(temp_results)     
    return results_df

In [37]:
qc_results = generate_results(post_processed_optimal_nodes)

In [38]:
random_combinations = []

 
for _ in range(1000):
    random_sample = []
    valid_sample = False
    while not valid_sample:
        for region in range(1, region_num + 1):
            #print((region -1)  * samples_per_region, region* samples_per_region-1)
            choice = random.randint((region -1)  * samples_per_region, region* samples_per_region-1)
            random_sample.append(choice)

        if tuple(random_sample) not in random_combinations:
            valid_sample = True
            random_combinations.append(tuple(random_sample))
            
random_results = generate_results(random_combinations)

In [39]:
from scipy.stats import ttest_ind

 
# Perform t-test
t_stat_color, p_value_color = ttest_ind(qc_results['color'], random_results['color'])
t_stat_shape, p_value_shape = ttest_ind(qc_results['shape'], random_results['shape'])
t_stat_component, p_value_component = ttest_ind(qc_results['components'], random_results['components'])
t_stat_score, p_value_score = ttest_ind(qc_results['score'], random_results['score'])
print(t_stat_color, p_value_color) 
print(t_stat_component, p_value_component)
print(t_stat_score, p_value_score)

-4.3904145421699265 1.237665071791334e-05
-9.235373766328967 1.2408235822794532e-19
-6.685973607781373 3.601773908775344e-11


In [40]:
# plot2d
import plotly.express as px
# Create a combined DataFrame for plotting
random_results['type'] = 'Random'
qc_results['type'] = 'Sampler'
combined_results = pandas.concat([random_results, qc_results])

# Scatter plot using Plotly
fig = px.scatter(combined_results, x='color', y='components', color='type', size='score',
                 hover_data=['name'], title='Comparison of Scores: Sampler vs Random',
                 labels={'color': 'Color Score', 'components': 'Component Score', 'score': 'Overall Score'})

fig.show()

In [41]:
#plot 3D

import plotly.graph_objects as go
# Create the figure
fig = go.Figure()

# Add traces for random_results
fig.add_trace(go.Scatter3d(
    x=random_results['color'],
    y=random_results['shape'],
    z=random_results['components'],
    mode='markers',
    marker=dict(
        size=5,
        color='#a0acbd',  # light blue
        opacity=0.6
    ),
    name="Random Sampling",
    showlegend=False
))

# Add traces for qc_results
fig.add_trace(go.Scatter3d(
    x=qc_results['color'],
    y=qc_results['shape'],
    z=qc_results['components'],
    mode='markers',
    marker=dict(
        size=5,
        color='#1993e0',  # dark blue
        opacity=1.0
    ),
    name="Quality Control Sampling",
    showlegend=False
))

# Create custom legend
legend_y_start = 1
legend_x = 1
for i, (label, color) in enumerate([
    ("Random Sampling", '#a0acbd'),
    ("QAOA Sampling", '#1993e0')
]):
    fig.add_trace(go.Scatter3d(
        x=[None], y=[None], z=[None],  # No actual data points
        mode='markers',
        marker=dict(size=10, color=color),  # Larger marker size for legend
        name=label,
        showlegend=True
    ))

# Update the layout
fig.update_layout(
    title={
        'text': '<b>Comparison of Sampling Methods<b>',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font_size': 24
    },
    scene=dict(
        xaxis=dict(
            backgroundcolor="rgb(0, 0, 0)",
            gridcolor="white",
            showbackground=True,
            zerolinecolor="white",
            title=dict(
                text="Color Score",
                font=dict(color="white")
            ),
            tickfont=dict(color="white"),
            autorange='reversed'
        ),
        yaxis=dict(
            backgroundcolor="rgb(0, 0, 0)",
            gridcolor="white",
            showbackground=True,
            zerolinecolor="white",
            title=dict(
                text="Shape Score",
                font=dict(color="white")
            ),
            tickfont=dict(color="white")
        ),
        zaxis=dict(
            backgroundcolor="rgb(0, 0, 0)",
            gridcolor="white",
            showbackground=True,
            zerolinecolor="white",
            title=dict(
                text="Component Score",
                font=dict(color="white")
            ),
            tickfont=dict(color="white")
        )
    ),
    legend=dict(
        title="Classification",
        x=0.9,
        y=0.8,
        xanchor='left',
        yanchor='top',
        font_size=18
    ),
    font=dict(
        color="White"
    ),
    paper_bgcolor='rgba(0,0,0)',
    plot_bgcolor='rgba(0,0,0)'
)

fig.show()

In [4]:
def generate_example_data(total_brick_num = 100,
                          regions = 6,
                          color_range = 10,
                          shape_range= 5,
                          component_range = 5):
    
    example_data = []
    brick_per_region = int(total_brick_num / regions)
    example_data.append("id,region,color,shape,components")
    _id = 1
    for region in range(1, regions+1):
        for brick in range(brick_per_region):
            example_data.append(
            f"{_id},{region},{random.randint(1,color_range)},{random.randint(1,shape_range)},{random.randint(1,component_range)}"
            )
            _id += 1
            
    with open(f"example_data_{total_brick_num}_{regions}.csv", "w") as fh:
        fh.write("\n".join(example_data))
generate_example_data(total_brick_num=18)